In [1]:
from datasets import load_dataset

In [2]:
dirs = """
cmg_ga_mu_corpusbeag
pmg_ga_co/RCPiarsachALL
nnc_ga_mu/CDNamedEntities01
nnc_ga_mu/CDAn_tOileanach_part3_newLTS
nnc_ga_mu/CDAn_tOileanach_part4_newLTS
nnc_ga_mu/CDNamedEntities02
nnc_ga_mu/CDAlphabet
nnc_ga_mu/CDAn_tOileanach_part2_newLTS
nnc_ga_mu/corpas_beag_newLTS
nnc_ga_mu/CDSaytime
nnc_ga_mu/CDAn_tOileanach_part1_newLTS
cmg_ga_mu/CMGOther
cmg_ga_mu/CMGCorpusBeag
anb_ga_ul/new_text02
anb_ga_ul/text01-004
anb_ga_ul/text01
anb_ga_ul/text01-001
anb_ga_ul/text02
anb_ga_ul/named-entities
nnc_en_ie
"""

In [3]:
dirl = [a for a in dirs.split("\n") if a]

In [4]:
script = "abairxml.py"

In [5]:
sets = {}
for dir in dirl:
    sets[dir] = load_dataset(script, data_dir=f"/home/jim/tmp/{dir}")

Using custom data configuration default-187ee4831f664a6d
Reusing dataset utterance_xml_dataset (/home/jim/.cache/huggingface/datasets/utterance_xml_dataset/default-187ee4831f664a6d/1.1.0/2ab4d5a39b76b2b213d176be1f94b8bb2c34198ea2fabf4b281e0784782b2d2b)
Using custom data configuration default-aa8abe91265ea5f3
Reusing dataset utterance_xml_dataset (/home/jim/.cache/huggingface/datasets/utterance_xml_dataset/default-aa8abe91265ea5f3/1.1.0/2ab4d5a39b76b2b213d176be1f94b8bb2c34198ea2fabf4b281e0784782b2d2b)
Using custom data configuration default-7667bb9aedbb818e
Reusing dataset utterance_xml_dataset (/home/jim/.cache/huggingface/datasets/utterance_xml_dataset/default-7667bb9aedbb818e/1.1.0/2ab4d5a39b76b2b213d176be1f94b8bb2c34198ea2fabf4b281e0784782b2d2b)
Using custom data configuration default-7a61ec2b352637f4
Reusing dataset utterance_xml_dataset (/home/jim/.cache/huggingface/datasets/utterance_xml_dataset/default-7a61ec2b352637f4/1.1.0/2ab4d5a39b76b2b213d176be1f94b8bb2c34198ea2fabf4b281e07

In [6]:
from pathlib import Path
def check_paths(batch):
    audio_path = Path(batch["audio"])
    return audio_path.is_file()

for dir in dirl:
    sets[dir]["train"] = sets[dir]["train"].filter(check_paths)

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

...

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [8]:
import torchaudio

def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["audio"])
    batch["speech"] = speech_array[0].numpy()
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["words"]
    return batch

for dir in dirl:
    sets[dir]["train"] = sets[dir]["train"].map(speech_file_to_array_fn, remove_columns=sets[dir]["train"].column_names)

  0%|          | 0/1453 [00:00<?, ?ex/s]

KeyError: 'audio'

In [9]:
import librosa
import numpy as np

def resample(batch):
    batch["duration"] = librosa.get_duration(y=batch["speech"], sr=batch["sampling_rate"])

    if batch["sampling_rate"] != 16_000:
        batch["speech"] = librosa.resample(np.asarray(batch["speech"]), batch["sampling_rate"], 16_000)
        batch["sampling_rate"] = 16_000

    return batch

for dir in dirl:
    sets[dir]["train"] = sets[dir]["train"].map(resample, num_proc=4)

In [10]:
sets

{'cmg_ga_mu_corpusbeag': DatasetDict({
     train: Dataset({
         features: ['speech', 'sampling_rate', 'target_text', 'duration'],
         num_rows: 1453
     })
 }), 'pmg_ga_co/RCPiarsachALL': DatasetDict({
     train: Dataset({
         features: ['speech', 'sampling_rate', 'target_text', 'duration'],
         num_rows: 1920
     })
 }), 'nnc_ga_mu/CDNamedEntities01': DatasetDict({
     train: Dataset({
         features: ['speech', 'sampling_rate', 'target_text', 'duration'],
         num_rows: 205
     })
 }), 'nnc_ga_mu/CDAn_tOileanach_part3_newLTS': DatasetDict({
     train: Dataset({
         features: ['speech', 'sampling_rate', 'target_text', 'duration'],
         num_rows: 1501
     })
 }), 'nnc_ga_mu/CDAn_tOileanach_part4_newLTS': DatasetDict({
     train: Dataset({
         features: ['speech', 'sampling_rate', 'target_text', 'duration'],
         num_rows: 1172
     })
 }), 'nnc_ga_mu/CDNamedEntities02': DatasetDict({
     train: Dataset({
         features: ['speech